# Localizacion de robot con EKF y balizas
Se dispone de un escenario donde se han desplegado en posiciones conocidas balizas estáticas que miden su distancia a un robot aéreo en dicho escenario. Se trata de desarrollar técnicas de estimación de posición 3D para un robot aéreo basados en filtro de Kalman extendido (EKF) que integra las medidas tomadas por las balizas. Plataforma de desarrollo: Matlab/ROS Lenguaje: Matlab o para ROS, C/C++ o Python

Steps:
1. Se dispone a realizar a interpretare el problema de triangulacion y trialateracion
1. Se dispone realizar 1 filtro de kalman 
1. se dispone de realizar el filtro de kalman extendido

Explicando las ecuaciones y todo al respecto

https://docs.ufpr.br/~danielsantos/ProbabilisticRobotics.pdf pagina 48

## Deteminacion de posicion utilizando balizas
obtener la posicion de un objeto de interes a partir de las distancias a posiciones de localizacion conocidadas. La localizacion basandose en la medicion de distancias a ciertos puntos de referencia de localizacion conocida (balizas ), para la localizacion de un objeto movil
<!--- [comment](http://platea.pntic.mec.es/rrodrigu/docs/MEMORIA_TRILATERACION_MEDIANTE_BIDETERMINANTE_CAYLEY_MENGER.pdf) -->

Para resolver el problema de localizacion  basta con resolver el sigiente sistema de ecuaciones que reporesental la solucion a la interseccion de las 3 esfereas definidas mediante sus centros en las posiciones de referencias y sus raidos como las distancias medidas al abjeto de interes.


$$\begin{matrix}
r_{2}^2 = (x_{1}-x_2)^2+ (y_{1}-y_2)^2 +(z_{1}-z_2)^2 \\
r_{3}^2 = (x_{1}-x_3)^2+ (y_{1}-y_3)^2 +(z_{1}-z_3)^2\\
r_{4}^2 = (x_{1}-x_4)^2+ (y_{1}-y_4)^2 +(z_{1}-z_4)^2
\end{matrix} $$


Dado que dicho sistema de escuaciones no es lineal, su solucion popuede resolver mediante metodos algebraicos, numericos como trilareacion o triangulacion, asi como mediante la linealizacion del sistema de ecuaciones como se realizara al realizar el filtro de kalman extendido.



El ruido de la baliza  es una  distribución normal,una distribucion estándar es aquella que tiene una media de 0 y una varianza de 1, 


In [65]:
from Beacon_EKF import Beacons_EKF
from Beacon_sim import Beacon_sim
import numpy as np

dt=0.2
init_pos = [2.5,3,2.0]

EKF = Beacons_EKF(step_time=dt,init_position=init_pos,init_speed=[0.0,0.0,0.0],q=.0099,r=0.001)
beacon = Beacon_sim(pos=init_pos,vel=[0.0,0.0,0.0],dt=dt,ruido =0.001)
#print(EKF.get_beacon_point())
#EKF.get_status()
#z = beacon.update_position()
#EKF.EKF.z =z
#print(EKF.EKF.z)
#z = beacon.update_position()
#EKF.EKF.z =z
#print(EKF.EKF.z)

In [69]:
xs, track_pos,br= [], [],[]
xs1 ,xs2, xs3 = [],[],[]
br1 ,br2, br3 = [],[],[]
track_posx,track_posy, track_posz=[],[],[]
for i in range (int(40/dt)):
    z = beacon.update_position()
    track_pos.append(beacon.get_pos().copy())
    br.append(z.copy())
    EKF.update(z=z)
    xs.append(EKF.get_status())
    EKF.predict()
    
time = np.arange(0, len(xs)*dt, dt)

for i in range(len(xs)):
    xs1.append(xs[i][0][0]) # valor de posicion x
    xs2.append(xs[i][1][0]) # valor de posicion y
    xs3.append(xs[i][2][0]) # valor de posicion z
    track_posx.append(track_pos[i][0])
    track_posy.append(track_pos[i][1])
    track_posz.append(track_pos[i][2])
    br1.append(br[i][0][0])
    br2.append(br[i][1][0])
    br3.append(br[i][2][0])
    #print(str(track_posx[i])+"--"+str(xs1[i]))
print("d")

In [70]:
%matplotlib notebook
import matplotlib.pyplot as plt

fig, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(15,5))
fig.suptitle('Horizontally stacked subplots')
ax1.plot(time, xs1,  label = "linex 1")
ax1.plot(time, track_posx, label = "linex 2")
ax1.set_title('posicion X')
ax2.plot(time, xs2,  label = "liney 1")
ax2.plot(time, track_posy, label = "liney 2")
ax2.set_title('posicion y')
ax3.plot(time, xs3,  label = "linez 1")
ax3.plot(time, track_posz, label = "linez 2")
ax3.set_title('posicion z')
#plt.plot(time, xs1,  label = "line 1")
#plt.plot(time, track_posx, label = "line 2")
plt.legend()
plt.show()

In [57]:
fig2, (ax11, ax21,ax31) = plt.subplots(1, 3,figsize=(15,5))
fig2.suptitle('Horizontally stacked subplots')
ax11.plot(time, br1,  label = "linex 1")
ax11.set_title('radio X')
ax21.plot(time, br2,  label = "liney 1")
ax21.set_title('distancia y')
ax31.plot(time, br3,  label = "linez 1")
ax31.set_title('distancia z')
#plt.plot(time, xs1,  label = "line 1")
#plt.plot(time, track_posx, label = "line 2")
plt.legend()
plt.show()

In [58]:
trayec_x =[]
trayec_y =[]
trayec_z =[]
init_pos = [2.5,3,2.0]

trayec_x.append(init_pos[0])
trayec_y.append(init_pos[1])
trayec_z.append(init_pos[2])

for i in range(0,50,1):
    trayec_x.append(trayec_x[-1])
    trayec_y.append(trayec_y[-1])
    trayec_z.append(trayec_z[-1]+0.1)
    
for i in range(0,50,1):
    trayec_x.append(trayec_x[-1])
    trayec_y.append(trayec_y[-1]+0.1)
    trayec_z.append(trayec_z[-1])
    
for i in range(0,50,1):
    trayec_x.append(trayec_x[-1]+0.1)
    trayec_y.append(trayec_y[-1])
    trayec_z.append(trayec_z[-1])
    
for i in range(0,50,1):
    trayec_x.append(trayec_x[-1])
    trayec_y.append(trayec_y[-1]-0.1)
    trayec_z.append(trayec_z[-1])
for i in range(0,50,1):
    trayec_x.append(trayec_x[-1]-0.1)
    trayec_y.append(trayec_y[-1])
    trayec_z.append(trayec_z[-1])
for i in range(0,50,1):
    trayec_x.append(trayec_x[-1])
    trayec_y.append(trayec_y[-1])
    trayec_z.append(trayec_z[-1]-0.1)

    
ax111 = plt.axes(projection = "3d")
ax111.plot_wireframe(np.array([trayec_x]),np.array([trayec_y]),np.array([trayec_z]))
plt.title("grafica 3d")

trayec = []
trayec.append(trayec_x)
trayec.append(trayec_y)
trayec.append(trayec_z)

plt.show()



In [59]:

EKF1 = Beacons_EKF(step_time=dt,init_position=init_pos,init_speed=[0.0,0.0,0.50],q=0.044,r= 0.1)#q=0.92-r=0,25
beacon1 = Beacon_sim(pos=init_pos,vel=[0,0.0,0.50],dt=dt,ruido=0.1)
beacon1.set_trayec(trayec)
beacon_base=EKF1.get_beacon_point()
print(beacon_base)
xs, track_pos,br= [], [],[]
xs1 ,xs2, xs3 = [],[],[]
br1 ,br2, br3 = [],[],[]
track_posx,track_posy, track_posz=[],[],[]
xs.append(EKF1.get_status())
#print(xs[-1])
for i in range(len(trayec[0]) -1):
    z = beacon1.update_position()
    track_pos.append(beacon1.get_pos().copy())
    br.append(z.copy())
    EKF1.update(z=z)
    xs.append(EKF1.get_status())
    EKF1.predict()
    
time = np.arange(0, (len(xs)-1)*dt, dt)

for i in range(len(xs)-1):
    xs1.append(xs[i][0][0]) # valor de posicion x
    xs2.append(xs[i][1][0]) # valor de posicion y
    xs3.append(xs[i][2][0]) # valor de posicion z
    track_posx.append(track_pos[i][0])
    track_posy.append(track_pos[i][1])
    track_posz.append(track_pos[i][2])
    br1.append(br[i][0][0])
    br2.append(br[i][1][0])
    br3.append(br[i][2][0])
print("finish")

In [60]:
fig, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(15,5))
fig.suptitle('Horizontally stacked subplots')
ax1.plot(time, xs1)
ax1.plot(time, track_posx, label = "linex 2")
ax1.set_title('posicion X')
ax2.plot(time, xs2,  label = "liney 1")
ax2.plot(time, track_posy, label = "liney 2")
ax2.set_title('posicion y')
ax3.plot(time, xs3,  label = "linez 1")
ax3.plot(time, track_posz, label = "linez 2")
ax3.set_title('posicion z')
#plt.plot(time, xs1,  label = "line 1")
#plt.plot(time, track_posx, label = "line 2")
plt.legend()
plt.show()

In [61]:

fig2, (ax11, ax21,ax31) = plt.subplots(1, 3,figsize=(15,5))
fig2.suptitle('Horizontally stacked subplots')
ax11.plot(time, br1,  label = "linex 1")
ax11.set_title('distancia beacon 1')
ax21.plot(time, br2,  label = "liney 1")
ax21.set_title('distancia beacon 2')
ax31.plot(time, br3,  label = "linez 1")
ax31.set_title('distancia beacon 3')
#plt.plot(time, xs1,  label = "line 1")
#plt.plot(time, track_posx, label = "line 2")
plt.legend()
plt.show()

In [62]:
ax111 = plt.axes(projection = "3d")
ax111.plot_wireframe(np.array([trayec_x]),np.array([trayec_y]),np.array([trayec_z]))
ax111.plot_wireframe(np.array([xs1]),np.array([xs2]),np.array([xs3]), color="red")
ax111.scatter(beacon_base[0][0], beacon_base[0][1], beacon_base[0][2], c='g', marker='o')
ax111.scatter(beacon_base[1][0], beacon_base[1][1], beacon_base[1][2], c='g', marker='o')
ax111.scatter(beacon_base[2][0], beacon_base[2][1], beacon_base[2][2], c='g', marker='o')
plt.title("grafica 3d")
print(beacon_base)
plt.show()